# Методы кластеризации для сегментации изображений

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Image, display
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from scipy.spatial.distance import pdist, squareform
from sklearn.cluster import SpectralClustering
import time
import os

In [ ]:
# ===============================
# Создание тестового изображения
# ===============================

def create_segmentation_test_image():
    """Создаёт тестовое изображение для сегментации"""

    print("📝 Создаём тестовое изображение...")

    img = np.zeros((400, 600, 3), dtype=np.uint8)

    # Фон - голубой
    img[:, :] = [200, 150, 100]

    # Красный объект
    cv2.circle(img, (150, 150), 80, (50, 50, 200), -1)
    cv2.rectangle(img, (50, 250), (250, 350), (50, 50, 200), -1)

    # Зелёный объект
    cv2.circle(img, (450, 100), 70, (50, 200, 50), -1)
    points = np.array([[350, 280], [450, 220], [550, 280], [500, 350]], np.int32)
    cv2.fillPoly(img, [points], (50, 200, 50))

    # Жёлтый объект
    cv2.rectangle(img, (300, 50), (550, 150), (0, 200, 200), -1)
    cv2.ellipse(img, (200, 350), (80, 40), 0, 0, 360, (0, 200, 200), -1)

    # Добавляем шум
    noise = np.random.randint(-20, 20, img.shape, dtype=np.int16)
    img = np.clip(img.astype(np.int16) + noise, 0, 255).astype(np.uint8)

    return img

img = create_segmentation_test_image()
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

plt.figure(figsize=(10, 6))
plt.imshow(img_rgb)
plt.title('Тестовое изображение для сегментации')
plt.axis('off')
plt.show()

In [ ]:
# ===============================
# ЗАДАНИЕ 1: K-Means кластеризация
# ===============================

def kmeans_manual(pixels, K=3, max_iter=100, epsilon=0.001):
    """
    Реализация K-Means "с нуля"

    Args:
        pixels: матрица пикселей (N×3 для RGB)
        K: количество кластеров
        max_iter: максимальное число итераций
        epsilon: порог сходимости

    Returns:
        labels: метки кластеров для каждого пикселя
        centers: центроиды кластеров
        inertia: сумма квадратов расстояний
    """

    N = len(pixels)

    # 1. Инициализация: случайно выбираем K пикселей как центроиды
    indices = np.random.choice(N, K, replace=False)
    centers = pixels[indices].astype(np.float32)

    print(f"🔄 Выполняем K-Means (K={K})...")

    for iteration in range(max_iter):
        # 2. Назначение: каждому пикселю - ближайший центроид
        distances = np.zeros((N, K))

        for k in range(K):
            distances[:, k] = np.sum((pixels - centers[k])**2, axis=1)

        labels = np.argmin(distances, axis=1)

        # 3. Обновление центроидов
        new_centers = np.zeros_like(centers)

        for k in range(K):
            cluster_pixels = pixels[labels == k]

            if len(cluster_pixels) > 0:
                new_centers[k] = np.mean(cluster_pixels, axis=0)
            else:
                # Если кластер пуст, генерируем новый случайный центроид
                new_centers[k] = pixels[np.random.randint(0, N)]

        # 4. Проверка сходимости
        center_shift = np.sum(np.sqrt(np.sum((new_centers - centers)**2, axis=1)))

        if iteration % 10 == 0:
            print(f"   Итерация {iteration}: сдвиг центроидов = {center_shift:.6f}")

        if center_shift < epsilon:
            print(f"   ✅ Сходимость на итерации {iteration}")
            break

        centers = new_centers

    # Вычисляем инерцию (WCSS)
    distances = np.zeros((N, K))
    for k in range(K):
        distances[:, k] = np.sum((pixels - centers[k])**2, axis=1)

    inertia = np.sum(np.min(distances, axis=1))

    return labels, centers, inertia

# Подготавливаем пиксели
pixels = img.reshape(-1, 3).astype(np.float32)

# Применяем K-Means для разных K
fig, axes = plt.subplots(2, 3, figsize=(18, 10))

wcss_values = []
K_values = range(2, 8)

for i, K in enumerate(K_values):
    labels, centers, inertia = kmeans_manual(pixels, K=K, max_iter=100)
    wcss_values.append(inertia)

    # Восстанавливаем сегментированное изображение
    centers = centers.astype(np.uint8)
    segmented = centers[labels]
    segmented_img = segmented.reshape(img.shape)

    # Визуализируем
    row, col = i // 3, i % 3
    axes[row, col].imshow(cv2.cvtColor(segmented_img, cv2.COLOR_BGR2RGB))
    axes[row, col].set_title(f'K = {K}')
    axes[row, col].axis('off')

    print(f"\nK = {K}:")
    print(f"   Инерция (WCSS): {inertia:.2e}")
    print(f"   Уникальные цвета: {len(np.unique(labels))}")

plt.suptitle('K-Means сегментация с разными K', fontsize=14)
plt.tight_layout()
plt.show()

# Метод "локтя" для выбора оптимального K
fig, ax = plt.subplots(figsize=(10, 6))

ax.plot(K_values, wcss_values, 'bo-', linewidth=2, markersize=8)
ax.set_xlabel('K (число кластеров)')
ax.set_ylabel('WCSS (инерция)')
ax.set_title('Метод "локтя" для выбора оптимального K')
ax.grid(True, alpha=0.3)

# Пытаемся найти "локоть"
slopes = np.diff(wcss_values) / np.diff(K_values)
elbow_idx = np.argmax(np.diff(slopes)) + 1
elbow_K = K_values[elbow_idx]

ax.axvline(elbow_K, color='r', linestyle='--', label=f'Рекомендуемый K = {elbow_K}')
ax.legend()

plt.tight_layout()
plt.show()

# Сравнение с OpenCV
print("\n🔄 Сравнение с OpenCV cv2.kmeans()...")

K = 4
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.2)

ret, labels_cv, centers_cv = cv2.kmeans(pixels, K, None, criteria, 10, cv2.KMEANS_PP_CENTERS)

centers_cv = centers_cv.astype(np.uint8)
segmented_cv = centers_cv[labels_cv.flatten()]
segmented_img_cv = segmented_cv.reshape(img.shape)

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

axes[0].imshow(img_rgb)
axes[0].set_title('Исходное изображение')
axes[0].axis('off')

axes[1].imshow(cv2.cvtColor(segmented_img_cv, cv2.COLOR_BGR2RGB))
axes[1].set_title(f'K-Means OpenCV (K={K})')
axes[1].axis('off')

plt.tight_layout()
plt.show()

# Визуализация кластеров в 3D RGB пространстве
fig = plt.figure(figsize=(12, 5))

# Исходные точки
ax1 = fig.add_subplot(121, projection='3d')
sample_idx = np.random.choice(len(pixels), 1000, replace=False)
ax1.scatter(pixels[sample_idx, 0], pixels[sample_idx, 1], pixels[sample_idx, 2],
           c=pixels[sample_idx]/255, s=1, alpha=0.3)
ax1.scatter(centers_cv[:, 0], centers_cv[:, 1], centers_cv[:, 2],
           c='red', s=200, marker='*', edgecolors='black', linewidth=2, label='Центроиды')
ax1.set_xlabel('R')
ax1.set_ylabel('G')
ax1.set_zlabel('B')
ax1.set_title('RGB пространство: пиксели и центроиды')
ax1.legend()

# Распределение пикселей по кластерам
ax2 = fig.add_subplot(122)
unique, counts = np.unique(labels_cv, return_counts=True)
colors = centers_cv.astype(np.uint8)[unique] / 255
bars = ax2.bar(unique, counts, color=colors)
ax2.set_xlabel('Кластер')
ax2.set_ylabel('Количество пикселей')
ax2.set_title('Распределение пикселей по кластерам')
ax2.set_xticks(unique)

plt.tight_layout()
plt.show()

In [ ]:
# ===============================
# ЗАДАНИЕ 2: Иерархическая кластеризация
# ===============================

def hierarchical_clustering_manual(pixels, K, method='ward'):
    """
    Упрощённая иерархическая кластеризация (агломеративная)

    Примечание: используем scipy для вычисления, но показываем концепцию
    """

    print(f"🔄 Выполняем иерархическую кластеризацию (метод: {method})...")

    # Выполняем иерархическую кластеризацию
    Z = linkage(pixels, method=method)

    # Получаем метки кластеров
    labels = fcluster(Z, K, criterion='maxclust') - 1

    return labels, Z

# Применяем иерархическую кластеризацию (на подмножестве для скорости)
sample_size = 2000
sample_idx = np.random.choice(len(pixels), sample_size, replace=False)
pixels_sample = pixels[sample_idx]

K = 4

print(f"Используем подмножество {sample_size} пикселей для демонстрации")

labels_hier, Z = hierarchical_clustering_manual(pixels_sample, K, method='ward')

# Визуализируем дендрограмму (на еще большем подмножестве для читаемости)
sample_size_dendro = 300
sample_idx_dendro = np.random.choice(len(pixels), sample_size_dendro, replace=False)
pixels_dendro = pixels[sample_idx_dendro]

Z_dendro = linkage(pixels_dendro, method='ward')

fig, ax = plt.subplots(figsize=(16, 8))

dendrogram(Z_dendro, ax=ax, leaf_font_size=8)
ax.axhline(y=50, color='r', linestyle='--', label='Порог для K=4')
ax.set_title('Дендрограмма иерархической кластеризации')
ax.set_xlabel('Индекс пикселя')
ax.set_ylabel('Расстояние')
ax.legend()

plt.tight_layout()
plt.show()

# Сравнение разных методов иерархической кластеризации
methods = ['ward', 'complete', 'average', 'single']
fig, axes = plt.subplots(2, 2, figsize=(14, 12))
axes = axes.flatten()

for i, method in enumerate(methods):
    Z = linkage(pixels_dendro, method=method)
    labels = fcluster(Z, K, criterion='maxclust') - 1

    # Отмечаем точки разных кластеров разными цветами
    ax = axes[i]
    colors = ['red', 'blue', 'green', 'orange', 'purple'][:K]

    for k in range(K):
        mask = labels == k
        ax.scatter(pixels_dendro[mask, 0], pixels_dendro[mask, 1],
                  c=colors[k], label=f'Кластер {k}', alpha=0.6, s=10)

    ax.set_title(f'Метод: {method.upper()}')
    ax.set_xlabel('R')
    ax.set_ylabel('G')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.suptitle('Сравнение методов иерархической кластеризации', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# ===============================
# ЗАДАНИЕ 3: Спектральная кластеризация
# ===============================

def spectral_clustering_image(pixels, K, sigma=1.0):
    """
    Спектральная кластеризация для изображения

    Args:
        pixels: матрица пикселей (N×3)
        K: количество кластеров
        sigma: параметр подобия в RBF ядре
    """

    print(f"🔄 Выполняем спектральную кластеризацию (K={K}, sigma={sigma})...")

    # Используем scikit-learn для спектральной кластеризации
    n_samples = min(len(pixels), 5000)  # Ограничиваем размер для скорости
    sample_idx = np.random.choice(len(pixels), n_samples, replace=False)
    pixels_sample = pixels[sample_idx]

    print(f"   Используем {n_samples} пикселей")

    # Спектральная кластеризация
    spectral = SpectralClustering(n_clusters=K, affinity='rbf', gamma=1/(2*sigma**2),
                                 random_state=42, n_init=10)
    labels_sample = spectral.fit_predict(pixels_sample)

    # Распространяем на все пиксели (используя ближайших соседей)
    from sklearn.neighbors import NearestNeighbors

    nn = NearestNeighbors(n_neighbors=1)
    nn.fit(pixels_sample)

    distances, indices = nn.kneighbors(pixels)
    labels = labels_sample[indices.flatten()]

    return labels, labels_sample, pixels_sample

# Применяем спектральную кластеризацию с разными параметрами sigma
sigmas = [1.0, 10.0, 30.0, 50.0]
fig, axes = plt.subplots(2, 2, figsize=(14, 12))
axes = axes.flatten()

K = 4

for i, sigma in enumerate(sigmas):
    labels, _, _ = spectral_clustering_image(pixels, K, sigma=sigma)

    # Раскрашиваем пиксели в цвета их кластеров
    colors_palette = np.array([[255, 0, 0], [0, 255, 0], [0, 0, 255], [255, 255, 0],
                               [255, 0, 255], [0, 255, 255]])

    colored = colors_palette[labels % len(colors_palette)]
    spectral_img = colored.reshape(img.shape)

    axes[i].imshow(spectral_img)
    axes[i].set_title(f'σ = {sigma}')
    axes[i].axis('off')

plt.suptitle('Спектральная кластеризация с разными σ', fontsize=14)
plt.tight_layout()
plt.show()

# Сравнение всех трёх методов
print("\n🔍 СРАВНЕНИЕ ТРЁХ МЕТОДОВ:")

K = 4

# K-Means
print("\n1. K-Means:")
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.2)
ret, labels_km, centers_km = cv2.kmeans(pixels, K, None, criteria, 10, cv2.KMEANS_PP_CENTERS)
centers_km = centers_km.astype(np.uint8)
segmented_km = centers_km[labels_km.flatten()].reshape(img.shape)

# Иерархическая
print("2. Иерархическая:")
Z = linkage(pixels, method='ward')
labels_hier_full = fcluster(Z, K, criterion='maxclust') - 1
centers_hier = np.array([np.mean(pixels[labels_hier_full == k], axis=0) for k in range(K)]).astype(np.uint8)
segmented_hier = centers_hier[labels_hier_full].reshape(img.shape)

# Спектральная
print("3. Спектральная:")
labels_spec, _, _ = spectral_clustering_image(pixels, K, sigma=20.0)
centers_spec = np.array([np.mean(pixels[labels_spec == k], axis=0) for k in range(K)]).astype(np.uint8)
segmented_spec = centers_spec[labels_spec].reshape(img.shape)

# Визуализируем
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

axes[0, 0].imshow(img_rgb)
axes[0, 0].set_title('Исходное изображение')
axes[0, 0].axis('off')

axes[0, 1].imshow(cv2.cvtColor(segmented_km, cv2.COLOR_BGR2RGB))
axes[0, 1].set_title('K-Means')
axes[0, 1].axis('off')

axes[1, 0].imshow(cv2.cvtColor(segmented_hier, cv2.COLOR_BGR2RGB))
axes[1, 0].set_title('Иерархическая')
axes[1, 0].axis('off')

axes[1, 1].imshow(cv2.cvtColor(segmented_spec, cv2.COLOR_BGR2RGB))
axes[1, 1].set_title('Спектральная')
axes[1, 1].axis('off')

plt.suptitle('Сравнение методов кластеризации (K=4)', fontsize=14)
plt.tight_layout()
plt.show()

# ДОМАШНЕЕ ЗАДАНИЕ

1. Реализуйте метод силуэта (silhouette score) для оценки качества кластеризации

2. Сравните производительность всех методов:
- Время выполнения
- Качество сегментации (валидация)
- Масштабируемость с размером изображения

3. Применяйте методы к реальным изображениям:
- Портреты (выделение кожи)
- Пейзажи (выделение неба, земли)
- Сложные сцены
